In [2]:
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Blogstate(TypedDict):
    title : str
    outline : str
    contant : str
    evaluate : int

In [4]:
model = ChatOllama(model='qwen3-coder:480b-cloud')

C:\Users\User\AppData\Local\Temp\ipykernel_12580\2776079175.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  model = ChatOllama(model='qwen3-coder:480b-cloud')


In [5]:
def create_outline(state : Blogstate) -> Blogstate:
    # fetch title
    title = state['title']
    
    # call llm gen outline
    prompt = f'Generate a deatiled outline for a blog on the topic - {title}'
    output = model.invoke(prompt).content
    
    # update state
    state['outline'] = output
    return state

In [6]:
def create_blog(state : Blogstate) -> Blogstate:
    title = state['title']
    outline = state['outline']
    
    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'
    output = model.invoke(prompt).content
    state['contant'] = output
    return state

In [7]:
def evaluate_blog(state : Blogstate) -> Blogstate:
    outline = state['outline']
    blog = state['contant']
    
    prompt = f'Based on this outtline -> \n{outline} \n Rate My Blog in integer value-> \n{blog}'
    output = model.invoke(prompt).content
    state['evaluate'] = output
    return state

In [11]:
graph = StateGraph(Blogstate)

# Nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# Edge
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [12]:
output = workflow.invoke({'title' : "Rise Of Ai"})

In [14]:
print(output['evaluate'])

## Blog Rating: **8/10**

### Strengths:
- **Comprehensive Coverage**: The blog thoroughly addresses all aspects of AI, from technical foundations to societal implications
- **Well-Structured**: Excellent organization with clear sections and logical flow
- **Current and Relevant**: Incorporates recent developments like ChatGPT and OpenAI
- **Balanced Perspective**: Addresses both benefits and challenges of AI
- **Actionable Insights**: Provides practical guidance for individuals and organizations
- **Strong Introduction and Conclusion**: Effective hook and compelling call-to-action

### Areas for Improvement:
- **Technical Depth**: Some sections could benefit from more detailed technical explanations for the target audience
- **Visual Elements**: Could incorporate charts, graphs, or infographics to illustrate key statistics
- **Case Studies**: Adding specific examples would enhance practical understanding
- **Interactive Elements**: Could benefit from links to additional resources or i